## 데이터 탐색의 목적
1. 데이터가 어떤 식으로 구성된지 확인하고 분석의 방향을 결정 
2. 사용자와 아이템의 특성을 파악하고 좋은 피쳐와 아이디어를 발굴 

In [ ]:
import pickle
import pandas as pd
import numpy as np
import os, sys, gc 
from plotnine import *
import plotnine

from tqdm import tqdm_notebook
import seaborn as sns
import warnings
import matplotlib.pyplot as plt
import matplotlib.font_manager as fm
import matplotlib as mpl
from matplotlib import rc
import re
from matplotlib.ticker import PercentFormatter
import datetime
from math import log # IDF 계산을 위해

In [ ]:
%config InlineBackend.figure_format = 'retina'
mpl.font_manager._rebuild()

fontpath = '../input/t-academy-recommendation/NanumBarunGothic.ttf'
font = fm.FontProperties(fname=fontpath, size=9).get_name()

mpl.pyplot.rc('font', family=font)
plt.rc('font', family=font)
plt.rcParams['font.family'] = font

In [ ]:
warnings.filterwarnings(action='ignore')

## 데이터 로드

In [ ]:
path = "../input/t-academy-recommendation/"
print(os.listdir(path))

### 데이터 스키마
![](https://img1.daumcdn.net/thumb/R1280x0/?scode=mtistory2&fname=https%3A%2F%2Fblog.kakaocdn.net%2Fdn%2FdkFzLS%2Fbtqyt68KeI6%2FAiLvkFGi9wHMnXu57VkcOk%2Fimg.png)

1. User 
2. contents 
3. magazine 
4. metadata
5. read

데이터의 종류는 크게 5가지로 분류가 가능합니다. 하지만, 메타 데이터의 magazine_id와 Magazine데이터의 id와 매핑되는 등 컬럼명이 조금 다른 점을 볼 수 있습니다. 해당 사항에 유의하면서 각 파일별로 어떠한 특성을 가지고 있는 지 살펴보겠습니다. 

In [ ]:
# pd.read_json : json 형태의 파일을 dataframe 형태로 불러오는 코드 
magazine = pd.read_json(path + 'magazine.json', lines=True) # lines = True : Read the file as a json object per line.
metadata = pd.read_json(path + 'metadata.json', lines=True)
users = pd.read_json(path + 'users.json', lines=True)

json 파일의 경우는 pd.read_json을 이용해서 쉽게 불러왔습니다. 하지만, read의 데이터는 파일명이 "시작일_종료일" 형태로 제공되었고 파일의 갯수도 3600개로 많습니다. 그렇기에 경로내에 있는 파일을 한번에 불러와서 하나의 파일로 합치는 작업이 필요합니다. 

In [ ]:
import itertools
from itertools import chain
import glob
import os 

input_read_path = path + '/read/'
# os.listdir : 해당 경로에 있는 모든 파일들을 불러오는 명령어 
file_list = os.listdir(input_read_path)
print(file_list[0:2])

In [ ]:
%%time 
read_df_list = []
exclude_file_lst = ['read.tar', '.2019010120_2019010121.un~']
for file in tqdm_notebook(file_list):
    # 예외처리 
    if file in exclude_file_lst:
        continue 
    else:
        file_path = input_read_path + file
        df_temp = pd.read_csv(file_path, header=None, names=['raw'])
        # file명을 통해서 읽은 시간을 추출(from, to)
        df_temp['from'] = file.split('_')[0]
        df_temp['to'] = file.split('_')[1]
        read_df_list.append(df_temp)
    
read_df = pd.concat(read_df_list)

In [ ]:
read_df.head()

In [ ]:
read_df['user_id'] = read_df['raw'].apply(lambda x: x.split(' ')[0])
read_df['article_id'] = read_df['raw'].apply(lambda x: x.split(' ')[1:])
read_df.head()

위와 같이 전처리한 경우에 하나의 user_id에 여러개의 article_id가 매핑됩니다. 분석의 편의상 하나의 user_id에 한개의 article_id가 매핑(1:1)되도록 article_id의 list를 풀어주는 작업을 진행합니다. 

In [ ]:
# 하나의 리스트로 반환하는 코드 
def chainer(s):
    return list(itertools.chain.from_iterable(s))

# article_id의 리스트가 풀어지면서 길어지는 것을 맞추기 위해서 np.repeat을 통해 같은 정보를 반복
read_cnt_by_user = read_df['article_id'].map(len)
read_rowwise = pd.DataFrame({'from': np.repeat(read_df['from'], read_cnt_by_user),
                             'to': np.repeat(read_df['to'], read_cnt_by_user),
                             'user_id': np.repeat(read_df['user_id'], read_cnt_by_user),
                             'article_id': chainer(read_df['article_id'])})

read_rowwise.reset_index(drop=True, inplace=True)

In [ ]:
del read_cnt_by_user
read_rowwise.head()

## 데이터 탐색 - Users

In [ ]:
users.head()

변수설명
- id : 사용자의 고유 식별코드 
- following_list : 내가 구독하고 있는 작가의 아이디 목록
- keyword_list : 독자가 아닌 작가일 경우에 작가의 글로 유입된 검색 키워드

In [ ]:
print("사용자의 수: ", users.shape[0])
print("작가의 수: ", users[users['keyword_list'].apply(lambda x: len(x)) != 0].shape[0])

사용자의 수는 30만명이고 그 중에서도 작가는 1만명으로 약 3.3%정도를 차지하고 있습니다. 

In [ ]:
print("구독하는 작가가 있는 사용자의 수: ", users[users['following_list'].apply(lambda x: len(x)) != 0].shape[0])
print("{}가 구독하는 작가가 있을 정도로 많은 비율을 차지".format('97.6%'))

In [ ]:
users[users['keyword_list'].apply(lambda x: len(x)) != 0].head(1)['keyword_list'].values[0][0:10]

사용자별로 구독하는 작가에 대해서 어떤 특징을 가지고 있는 지 분석을 시작해보겠습니다. 기본적인 아이디어로는 아래와 같습니다. 
1. 사용자별로 구독하는 작가의 수는 얼마나 되는지? 
2. 어떤 작가를 주로 구독하는지? 
3. 주로 유입되는 키워드는 무엇인지? 

In [ ]:
users['following_count'] = users['following_list'].apply(lambda x: len(x))

In [ ]:
following_cnt_by_id = pd.DataFrame(users.groupby('following_count')['id'].count()).reset_index()

(ggplot(data=following_cnt_by_id)
    + geom_point(aes(x='following_count', y='id'), colour = '#49beb7')
    + theme_minimal()
    + ggtitle("구독하는 작가의 수별 사용자의 수")
    + labs(x="구독하는 작가의 수", y="사용자 수") 
    + theme(text = element_text(fontproperties=fm.FontProperties(fname=fontpath, size=9)),
         axis_text_x = element_text(angle=60, color='black'),
         axis_text_y = element_text(color='black'),
         figure_size=(8,4))
 )

In [ ]:
pd.DataFrame(users['following_count'].describe()).T

구독작가의 수의 중위수는 2명으로 분포가 극단적으로 왼쪽으로 치우친 형태인 것을 볼 수 있습니다. 이번에는 반대로 사람들이 어떤 작가를 구독하는 지 살펴보겠습니다. 

In [ ]:
following_cnt_by_user = users['following_list'].map(len)
following_rowwise = pd.DataFrame({'user_id': np.repeat(users['id'], following_cnt_by_user),
                             'author_id': chainer(users['following_list'])})

following_rowwise.reset_index(drop=True, inplace=True)

In [ ]:
following_rowwise.head()

In [ ]:
following_cnt_by_id = following_rowwise.groupby('author_id')['user_id'].agg({'count'}).reset_index().sort_values(by='count', ascending=False)
following_cnt_by_id.head(10).T

상위 10명의 작가를 보면 1등인 brunch는 공식사이트로 가장 많은 구독자의 수를 보유하고 있고 2등부터는 차이가 크게 변하는 것을 볼 수 있습니다. 

In [ ]:
following_cnt_by_id_ = following_cnt_by_id[following_cnt_by_id['author_id'] != '@brunch']
(ggplot(data=following_cnt_by_id_)
    + geom_histogram(aes(x='count', y='stat(count)'), fill = '#49beb7', binwidth=10)
    + theme_minimal()
    + ggtitle("작가별로 평균 구독자의 수")
    + labs(x="평균 구독자의 수", y="빈도") 
    + theme(text = element_text(fontproperties=fm.FontProperties(fname=fontpath, size=9)),
         axis_text_x = element_text(angle=60, color='black'),
         axis_text_y = element_text(color='black'),
         axis_line=element_line(color="black"),
         axis_ticks=element_line(color = "grey"),
         figure_size=(8,4))
 )

In [ ]:
pd.DataFrame(following_cnt_by_id['count'].describe()).T

In [ ]:
del following_cnt_by_id_
gc.collect()

실제로 히스토그램을 살펴보면 대부분의 값이 왼쪽에 치우친 롱테일형태의 데이터임을 알 수 있습니다. 

In [ ]:
keyword_dict = {}
for i in tqdm_notebook(users[users['keyword_list'].apply(lambda x: len(x)) != 0]['keyword_list'].values):
    for j in range(0, len(i)):
        word = i[j]['keyword']
        cnt = i[j]['cnt']
        try:
            keyword_dict[word] += cnt
        except:
            keyword_dict[word] = cnt

In [ ]:
# wordcloud에 대한 자세한 정보는 lovit님의 블로그 https://lovit.github.io/nlp/2018/04/17/word_cloud/를 참고하시기 바랍니다. 
from wordcloud import WordCloud
from PIL import Image

wordcloud = WordCloud(
    font_path = fontpath,
    width = 800,
    height = 800,
    background_color="white",
    mask= np.array(Image.open(path + "/figure/RS-KR.png"))

)
wordcloud = wordcloud.generate_from_frequencies(keyword_dict)

In [ ]:
fig = plt.figure(figsize=(12, 8))
plt.imshow(wordcloud, interpolation="bilinear")
plt.show()
fig.savefig('wordcloud.png')

In [ ]:
del users
del wordcloud
del keyword_dict
gc.collect()

## 데이터 탐색 - Read

In [ ]:
read_rowwise.head()

변수설명
- from : 시작일 
- to : 종료일
- user_id : 사용자의 식별코드
- article_id : 작품의 식별코드

데이터를 보면 article_id에 결측치가 있는 경우가 있습니다. 해당 결측치는 본문의 글이 삭제되거나 작가가 탈퇴하는 등의 이슈로 발생한 문제입니다. 이러한 article_id는 추천에 영향을 주지 않도록 전처리를 통해 제거해줍니다. 

In [ ]:
# article_id가 없는 경우 삭제 
read_rowwise = read_rowwise[read_rowwise['article_id'] != ''].reset_index(drop=True)

In [ ]:
# 읽은날짜와 시간 추출 
read_rowwise['dt'] = read_rowwise['from'].astype(str).apply(lambda x: x[0:8]).astype(int)
read_rowwise['hr'] = read_rowwise['from'].astype(str).apply(lambda x: x[8:10]).astype(int)
read_rowwise['read_dt'] = pd.to_datetime(read_rowwise['dt'].astype(str).apply(lambda x: x[0:4] + '-' + x[4:6] + '-' + x[6:8]))

In [ ]:
read_rowwise['article_id'].value_counts()[0:5]

In [ ]:
following_cnt_by_id.head(10).T

In [ ]:
del following_cnt_by_id
gc.collect()

사람들이 자주 읽는 글의 목록을 살펴보면 brunch의 글이 3개나 있는 것을 볼 수 있습니다. 또한, 구독자가 많은 상위 10명의 작가에 해당하는 글이 4건이나 있는 것을 볼 수 있습니다. 실제로 구독자가 많은 순위와 얼마나 비슷한지 살펴보기 위해서 사람들이 많이 글을 읽는 상위 10명의 작가를 보겠습니다. 

In [ ]:
read_rowwise['author_id'] = read_rowwise['article_id'].apply(lambda x: str(x).split('_')[0])
pd.DataFrame(read_rowwise['author_id'].value_counts()).head(10).T

구독자가 많은 상위 10건과 읽는 사용자가 많은 상위 10건을 보면 의외로 brunch와 tenbody 두개만 겹치는 것을 볼 수 있습니다. 이에 대해서 좀 더 자세히 살펴보기 위해서 사용자들이 구독하는 작가의 글을 얼마나 읽는 지 살펴보겠습니다. 

In [ ]:
# 구독하는 작가의 글을 읽는 비율 vs 그렇지 않은 작가의 글을 읽는 비율 
following_rowwise['is_following'] = 1
read_rowwise = pd.merge(read_rowwise, following_rowwise, how='left', on=['user_id', 'author_id'])

In [ ]:
del following_rowwise
gc.collect()

In [ ]:
read_rowwise['is_following'] = read_rowwise['is_following'].fillna(0)
read_rowwise['is_following'].value_counts(normalize=True)

In [ ]:
read_following_author = read_rowwise.groupby(['user_id'])['is_following'].agg({'mean'}).reset_index()

(ggplot(data=read_following_author)
    + geom_histogram(aes(x='mean', y='stat(count)'), fill = '#49beb7', binwidth=0.005)
    + theme_minimal()
    + ggtitle("사용자별 구독하는 작가의 글을 읽는 비율")
    + labs(x="글 소비 비율", y="빈도") 
    + theme(text = element_text(fontproperties=fm.FontProperties(fname=fontpath, size=9)),
         axis_text_x = element_text(angle=60, color='black'),
         axis_text_y = element_text(color='black'),
         axis_line=element_line(color="black"),
         axis_ticks=element_line(color = "grey"),
         figure_size=(8,4))
 )

In [ ]:
del read_following_author
gc.collect()

글의 소비 비율에 따른 빈도수를 살펴보면 0과 1에 극단적으로 치우친 형태임을 알 수 있습니다. **추천시에 글 소비 비율이 1에 가까운 사용자에게는 구독하는 작가의 글을 위주로 추천을 해주고 글 소비 비율이 0에 가까운 사용자에게는 전체 작가의 글을 다양하게 추천해주는 게 좋을 것이라는 인사이트를 얻을 수 있습니다.**

글 소비 비율이 0에 가까운 사용자의 경우 왜 그러한 행동을 보이는 지 살펴보도록 하겠습니다. 

첫번째로 발견한 이유는 애초에 글을 조금만 읽었을 가능성이 있습니다. 실제로 글 소비 비율이 1인 사용자 또한 동일한 문제를 가지고 있을 수도 있습니다. 

참고자료 
- 브런치 데이터의 탐색과 시각화 : https://brunch.co.kr/@kakao-it/332
- Kakao Arena 2회 대회 : 브런치 사용자를 위한 글 추천 대회 : https://arena.kakao.com/forum/topics/10